Performances of the agents. 
1. Log into wandb api 
2. Get the list of runs 
3. For each run, get the summary dict and the analysis df + eval df 
4. Skip the second part for now 
5. build and then group the results by
    Multiindex,
    Rows - tuple 
        i. algorithm type 
        ii. use rnn

    Columns - 
        i. R_type_1
        ii. R_type 2 
    
    For each 
        1. steps 
        2. rewards 
        3. assists 

In [2]:
WANDB_PROJECT = "rllib5"
MAX_EPISODES = 25000

In [3]:
# imports and connect to wandb -> 'runs' is a list of runs
import wandb 
import pandas as pd
import json
import numpy as np
api = wandb.Api()
entity, project = "tpn", WANDB_PROJECT
all_runs = api.runs(entity + "/" + project)

runs = []
total_runs = finished_runs = failed_runs = ongoing_runs = 0
for run in all_runs:
    total_runs += 1
    if run.state == "finished":
        runs.append(run)
        finished_runs += 1
    elif run.state == "crashed":
        failed_runs += 1
    elif run.state == "running":
        ongoing_runs +=1 
print(f"""
Total runs: {total_runs}
Finished runs: {finished_runs}
Failed runs: {failed_runs}
Ongoing runs: {ongoing_runs}
""")



Total runs: 56
Finished runs: 24
Failed runs: 32
Ongoing runs: 0



In [7]:
# Performanace DF 1 - all runs; check top for df structure
from random import sample

name_list = []
runs_summary_df = pd.DataFrame(
    [],
    index=np.arange(len(runs)),
    columns=[
        "algorithm_type",
        "use_lstm",
        "reward_type",
        "t_steps",
        "t_assists",
        "t_rewards",
        "e_steps",
        "e_assists",
        "e_rewards",
    ],
)

x_series = list(range(0, MAX_EPISODES, 100))
runs_performance_df = pd.DataFrame(
    [], index=x_series, columns=pd.MultiIndex.from_tuples([], names=["name", "type"])
)
sample_df = pd.DataFrame([], index=np.arange(0, MAX_EPISODES + 1, 100)).fillna(0)

for i, run in enumerate(runs):
    name_list.append(run.name)
    # runs_performance_df.loc[(run.config['training']["model"]["use_lstm"], run.config["algorithm_type"]),:] = \
    # [1, 1, 1, i, i, i]
    runs_summary_df.loc[i] = [
        run.config["algorithm_type"],
        run.config["training"]["model"]["use_lstm"],
        run.config["env_config"]["reward_type"],
        0,
        0,
        0,
        0,
        0,
        0,
        # run.summary["t_steps"],
        # run.summary["t_assists"],
        # run.summary["t_rewards"],
        # run.summary["e_steps"],
        # run.summary["e_assists"],
        # run.summary["e_rewards"],
    ]
    run_history_df = pd.DataFrame(run._full_history())
    run_history_df.set_index("episodes_total", inplace=True)
    sample_df = sample_df.merge(
        run_history_df[["episode_reward_mean", "episode_len_mean"]],
        how="outer",
        left_index=True,
        right_index=True,
    )
    sample_df = sample_df.T
    sample_df.sort_index(axis=1, inplace=True)
runs_history_df = pd.DataFrame(
    [],
    index = pd.MultiIndex.from_product(
        [[True, False],['independent', 'centralized', 'shared']],
        names = ["use_rnn", "agent_type"]
    ),
    columns = pd.MultiIndex.from_product(
        [["type_1", "type_2"], ["steps", "reward", "assits"]],
        names = ["reward_type", "metric (cumulative)"])
).fillna(0.0)
    # sample_df.fillna(method="ffill", inplace=True)

print(f"Number of runs: {len(name_list)}")
# print(runs_summary_df)

MergeError: Passing 'suffixes' which cause duplicate columns {'episode_reward_mean_x', 'episode_len_mean_x'} is not allowed.

In [31]:
gropued_df = runs_summary_df.groupby(["algorithm_type", "use_lstm", 'reward_type']).mean()
runs_performance_df = pd.DataFrame(
    [],
    index = pd.MultiIndex.from_product(
        [[True, False],['independent', 'centralized', 'shared']],
        names = ["use_rnn", "agent_type"]
    ),
    columns = pd.MultiIndex.from_product(
        [["type_1", "type_2"], ["steps", "reward", "assits"]],
        names = ["reward_type", "metric (cumulative)"])
).fillna(0.0)
for i, row in gropued_df.iterrows():
    runs_performance_df.loc[(i[1], i[0]), (i[2], "steps")] = row["t_steps"]
    runs_performance_df.loc[(i[1], i[0]), (i[2], "reward")] = row["t_rewards"]
    runs_performance_df.loc[(i[1], i[0]), (i[2], "assits")] = row["t_assists"]
print(runs_performance_df)


reward_type         type_1               type_2              
metric (cumulative)  steps reward assits  steps reward assits
use_rnn agent_type                                           
True    independent    0.0    0.0    0.0    0.0    0.0    0.0
        centralized    0.0    0.0    0.0    0.0    0.0    0.0
        shared         0.0    0.0    0.0    0.0    0.0    0.0
False   independent    0.0    0.0    0.0    0.0    0.0    0.0
        centralized    0.0    0.0    0.0    0.0    0.0    0.0
        shared         0.0    0.0    0.0    0.0    0.0    0.0


In [34]:
# performance graphs data load from csvs and plot   
steps_df = pd.read_csv("/home/priyam/projects/predator-prey/temp/wandb_export_2023-12-10T18 32 01.717+11 00.csv")
# matplolib make figure 
# 2x3 subplots
# 1. subplot 1.[1-3] - steps, rewards, assists vs epsiodes (without rnns)
# 2. subplot 1.[4-6] - steps, rewards, assists vs epsiodes (with rnns)
# for each subplot 3 lines - independent, centralized, shared
steps_df.describe()

,episodes_total,training.model.use_lstm: true - _step,training.model.use_lstm: true - _step__MIN,training.model.use_lstm: true - _step__MAX,training.model.use_lstm: true - episode_len_mean,training.model.use_lstm: true - episode_len_mean__MIN,training.model.use_lstm: true - episode_len_mean__MAX
count,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000,1746.000000
mean,8330.038946,72.928599,72.855097,73.002291,69.259543,68.966100,69.538288
std,4208.590735,33.811555,33.765298,33.863979,12.037706,12.008207,12.188978
min,2.000000,0.000000,0.000000,0.000000,46.150000,46.150000,46.150000
25%,4656.500000,44.000000,44.000000,44.000000,59.856250,59.402500,59.982500
50%,8190.500000,73.000000,73.000000,73.000000,66.310000,66.065000,66.590000
75%,11914.750000,102.000000,102.000000,102.000000,76.620000,76.120000,77.130000
max,16523.000000,135.000000,135.000000,136.000000,101.000000,101.000000,101.000000
